In [1]:
import pandas as pd
import numpy as np
import sys
import multiprocessing
import time
import os
import matplotlib.pyplot as plt
import seaborn as sns
from dateutil.parser import parse
from pandas.tseries.offsets import BDay
import pickle as pkl
import fnmatch

# extra imports
from collections import OrderedDict
from itertools import zip_longest
import datetime as dt
from datetime import timedelta

import glob
import fathon
from fathon import fathonUtils as fu


import matplotlib as mpl
from multiprocessing import Pool, freeze_support, Manager

import itertools
colormap = plt.cm.RdBu
plt.style.use(os.path.join(mpl.get_configdir(),'latexstyle.mplstyle'))
from collections import defaultdict
# import freqopttest.kernel as kernel

In [113]:
def linRangeByStep(start, end, step=1):
    """Array of linearly separated elements.
    
    Parameters
    ----------
    start : int
        Smallest element.
    end : int
        Biggest element.
    step : int
        Step between two consecutive elements (default : 1).
    Returns
    -------
    numpy ndarray
        Array of linearly separated elements.
    """
    return np.arange(start, end + 1, step, dtype=np.int64)

def linRangeByCount(start, end, count=-1):
    """Array of linearly separated elements.
    
    Parameters
    ----------
    start : int
        Smallest element.
    end : int
        Biggest element.
    count : int
        Number of elements (default : `end` - `start` + 1).
    Returns
    -------
    numpy ndarray
        Array of linearly separated elements.
    """
    if count == -1 or count > (end - start + 1):
        count = end - start + 1
        
    return np.linspace(start, end, count, endpoint=True, dtype=np.int64)

In [2]:
import os
os.listdir('/home/ak/.config/matplotlib/')

['latexstyle.mplstyle']

In [3]:
t71 = '/media/ak/T7/August11th2022Experiments/'
expInputFiles = os.path.join(t71,'ExperimentInputFiles' )
expOne = os.path.join(t71, 'ExperimentOne')
figuresDestination = '/home/ak/Documents/Papers/StylisedFactsPaper/figures/'
n =[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79]

In [4]:
YM1 = os.path.join(expOne,'YM1')
tick_mfdfa_files =sorted([f for f in os.listdir(YM1) if str('tick') in f])
len(tick_mfdfa_files)

146

In [5]:
idx = 10
tickIdxMfdfaFileLoc = os.path.join(YM1, tick_mfdfa_files[idx])


In [6]:
pd.read_pickle(tickIdxMfdfaFileLoc)['micro_price_change']

0        0.000000e+00
1       -1.331701e-07
2       -1.745295e-07
3       -3.448398e-07
4        6.752016e-08
             ...     
82454   -1.392161e-07
82455    1.603283e-07
82456    5.294183e-09
82457    8.222602e-06
82458   -1.733489e-08
Name: pct_change_micro_price, Length: 82459, dtype: float64

In [7]:
micro_variables = ['arrival_rates', 'gk_vol', 'median_traded_volume', 'micro_price_change']

gk_vol_dict = defaultdict(dict)

arrival_rates_dict = defaultdict(dict)
median_traded_volume_dict = defaultdict(dict)
micro_price_change_dict = defaultdict(dict)
mfdfa_dict = defaultdict(dict)
tau_dict = defaultdict(dict)
alpha_dict = defaultdict(dict)
mfSpect_dict = defaultdict(dict)
list_H_intercept_dict = defaultdict(dict)
list_H_dict = defaultdict(dict)
n_F_dict = defaultdict(dict)


for idx in range(0, 146):
    tickIdxMfdfaFileLoc = os.path.join(YM1, tick_mfdfa_files[idx])
    pkl_dict = pd.read_pickle(tickIdxMfdfaFileLoc)
    keys = sorted(list(pd.read_pickle(tickIdxMfdfaFileLoc).keys()))
    bar = list(set(keys).difference(set(micro_variables)))[0]
    gk_vol_dict[idx][bar] = pkl_dict['gk_vol']
    median_traded_volume_dict[idx][bar] = pkl_dict['median_traded_volume']
    arrival_rates_dict[idx][bar] = pkl_dict['arrival_rates']
    micro_price_change_dict[idx][bar] = pkl_dict['micro_price_change']
    tau_dict[idx][bar] = pkl_dict[str(bar)]['tau']
    alpha_dict[idx][bar] = pkl_dict[str(bar)]['alpha']
    mfSpect_dict[idx][bar] = pkl_dict[str(bar)]['mfSpect']
    n_F_dict[idx][bar] = pkl_dict[str(bar)]['n_F']
    list_H_dict[idx][bar] = pkl_dict[str(bar)]['list_H']
    list_H_intercept_dict[idx][bar] = pkl_dict[str(bar)]['list_H_intercept']
    n_F_dict[idx][bar] = pkl_dict[str(bar)]['n_F']
    
    
    
    
    
                       
    


In [8]:
# micro_price_change_dict

In [141]:
name='dollar'
d =micro_price_change_dict



In [10]:
# get all second level keys of all first level keys together (set comprehension)
sorted(list({k2 for v in d.values() for k2 in v}))

['dollar', 'tick', 'volume']

In [11]:
# get all first level keys

#list(d.keys())

In [12]:
# get all possible pairs of  (first, second) keys
tick_all_pairs=[(k1,k2) for (k1, v) in d.items() for k2 in v if k2=='tick' ]
volume_all_indices=[(k1) for (k1, v) in d.items() for k2 in v if k2=='volume' ]
tick_all_indices=[(k1) for (k1, v) in d.items() for k2 in v if k2=='tick' ]

In [99]:
# sorted(list({k2 for v in d.values() for k2 in v}))

In [155]:

micro_price_change = dict()
mfSpect_bar_choice = dict()
alpha_bar_choice = dict()
gk_vol = dict()

for f in tick_all_indices:
    micro_price_change[f] = d[f]['tick']
    mfSpect_bar_choice[f] = mfSpect_dict[f]['tick'] 
    alpha_bar_choice[f] = alpha_dict[f]['tick'] 
    gk_vol[f] = gk_vol_dict[f]['tick']

In [143]:
# for f in tick_all_indices:
#     print(d[f]['tick'])
mf_spect_df = pd.DataFrame.from_dict(mfSpect_bar_choice).dropna(0)
alpha_df = pd.DataFrame.from_dict(alpha_bar_choice).dropna(0)

print(mf_spect_df.shape, alpha_df.shape)
#rows x columns

(38, 57) (38, 57)


In [147]:
df = pd.DataFrame.from_dict(micro_price_change).dropna()
median_all_pct_change= pd.DataFrame(df.median(axis=0))
median_all_pct_change.shape
len(list(df.columns))

57

In [160]:
mean = df.mean(axis=0)
first_regime = np.quantile(median.values, 0.33)
second_regime = np.quantile(median.values, 0.66)

In [163]:
median.describe()

count    57.000000
mean     -0.000190
std       0.000042
min      -0.000211
25%      -0.000211
50%      -0.000211
75%      -0.000211
max      -0.000105
dtype: float64

In [122]:
# df['regime'], b = pd.qcut(df['median'].values, q = [0, 0.33, 0.66, 1], duplicates = 'drop',retbins = True)
# # c, b = pd.qcut(df['median'].values, q = [0, 0.25, 0.5, 0.75, 1], duplicates = 'drop',retbins = True)

57

In [192]:
regime_dict=dict()
for locIdx in range(0, df.shape[1]):
    columnDf = df.iloc[:,locIdx]
    c, b = pd.qcut(columnDf.values, q = [0, 0.33, 0.66, 1], labels = [1,2,3],  duplicates = 'drop',retbins = True)
    regime_dict[locIdx] = c

In [224]:
regime_df = pd.DataFrame(regime_dict)
regimes_count = dict()
for i in range(0,regime_df.shape[1]):
    regimes_count[i] = (regime_df.iloc[i].value_counts())


In [227]:
regime_counts_df = pd.DataFrame(regimes_count)

In [230]:

regime_counts_df

,0,1,2,3,4,5,6,7,8,9,...,47,48,49,50,51,52,53,54,55,56
1,0,41,30,41,11,24,0,13,13,22,...,0,13,0,13,0,41,13,11,24,0
2,57,16,16,5,16,16,16,27,27,5,...,22,5,27,44,16,5,16,18,5,18
3,0,0,11,11,30,17,41,17,17,30,...,35,39,30,0,41,11,28,28,28,39
